# Spørring som både gir tekst(navn) og kode(ID) fra JSON-stat
### eksempel HS-koder utenrikshandel

### Importerer biblioteker
Bruker [pyjstat](https://pypi.org/project/pyjstat/) og pandas. 

In [1]:
from pyjstat import pyjstat  
import pandas as pd  # Pandas lastes forsåvidt som del av pyjstat
import requests

Import / eksport alle varekoder til US siste måned, ca. 65000 celler. Max grense for en spørring i PxWebApi er 800.000 celler, inkl tomme celler.

### URL med tabellens metadata

GET_URL = https://data.ssb.no/api/pxwebapi/v2/tables/08801/data?lang=no&valueCodes[Varekoder]=*&valueCodes[ImpEks]=1,2&valueCodes[ContentsCode]=Mengde1,Verdi,Mengde2&valueCodes[Tid]=2024&codelist[Varekoder]=vs_VareKoderKN8Siff&codelist[Land]=vs_LandKoderAlf1

JSON-stat2 er default ut-format. En kan også legge på andre med `&outputFormat=` som csv eller xlsx. 

Det er greit å være klar over at URLer APIet *ikke* er følsomt for store og små bokstaver.

Viser de enkelte elementene. codelist elementene er i de fleste tilfellene overflødige/redundante og kan tas vekk. 


In [2]:
# De enkelte elementene i spørringen

tabid = "08799" # kan endres til årstabellen 08801, uten andre endringer
lang = "lang=no" # language code can also be "en"
get_stem = "https://data.ssb.no/api/pxwebapi/v2/tables/" + tabid + "/data?" + lang   # 'https://data.ssb.no/api/v0/no/table/08799'
get_query = "&valueCodes[Varekoder]=*\
&valueCodes[ImpEks]=1,2\
&valueCodes[Land]=US\
&valueCodes[ContentsCode]=Mengde1,Verdi,Mengde2\
&valueCodes[Tid]=top(1)"

Spørring, kan tas fra konsoll: Import / eksport alle varekoder til US siste måned, ca. 65000 celler. Max grense for en spørring i PxWebApi er 800.000 celler, inkl tomme celler

In [3]:
GET_URL = get_stem + get_query 

In [4]:
res = requests.get(GET_URL)

http kode 200 er OK

In [5]:
res

<Response [200]>

### Leser JSON-stat resultatet med biblioteket pyjstat
Lagrer det som datasett ds.

In [6]:
ds = pyjstat.Dataset.read(res.text)

In [7]:
type(ds)

pyjstat.pyjstat.Dataset

Se datasettet ds

In [8]:
# ds

### Henter noen overordnete metadata fra JSON-stat datasettet

In [9]:
tittel = ds['label']
print(tittel)

08799: Utenrikshandel med varer, etter varenummer, import/eksport, land, statistikkvariabel og måned


Sist Oppdatert som GMT

In [10]:
last_update = ds['updated']
print(last_update)

2025-10-15T06:00:00Z


Henter kilde

In [11]:
source = ds['source']
print(source)

Statistisk sentralbyrå


Roles gir noen snarveier til IDene på faste dimensjoner i datasettet

In [12]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'geo': ['Land'], 'metric': ['ContentsCode']})


### Lager dataframes
Vi må lage to dataframes, en med tekst og en med ID. Pyjstat tillater bare enten/eller

In [13]:
hstrade = ds.write('dataframe')
hstrade_id = ds.write('dataframe', naming='id')

In [14]:
hstrade.head()

,varenummer,import/eksport,land,statistikkvariabel,måned,value
0,"Uspesifiserte varer, tollbare (1988-2003) (M1=...",Import,USA,Mengde 1 (M1),2025M09,0
1,"Uspesifiserte varer, tollbare (1988-2003) (M1=...",Import,USA,Verdi (kr),2025M09,0
2,"Uspesifiserte varer, tollbare (1988-2003) (M1=...",Import,USA,Mengde 2 (M2),2025M09,0
3,"Uspesifiserte varer, tollbare (1988-2003) (M1=...",Eksport,USA,Mengde 1 (M1),2025M09,0
4,"Uspesifiserte varer, tollbare (1988-2003) (M1=...",Eksport,USA,Verdi (kr),2025M09,0


In [15]:
hstrade_id.head()

,Varekoder,ImpEks,Land,ContentsCode,Tid,value
0,00000011_1988,1,US,Mengde1,2025M09,0
1,00000011_1988,1,US,Verdi,2025M09,0
2,00000011_1988,1,US,Mengde2,2025M09,0
3,00000011_1988,2,US,Mengde1,2025M09,0
4,00000011_1988,2,US,Verdi,2025M09,0


### Lager en ny kolonne med både HS varekode og tekst slått sammen

In [16]:
hstrade['hstrade_combi'] = hstrade_id['Varekoder'] + ' ' + hstrade['varenummer']

In [17]:
hstrade.columns

Index(['varenummer', 'import/eksport', 'land', 'statistikkvariabel', 'måned',
       'value', 'hstrade_combi'],
      dtype='object')

### Lager en ny dataframe som bare har med kolonnene vi ønsker, i ny rekkefølge. 
Merk doble [[ ]] for å lage dataframe

In [18]:
hstrade_new = hstrade[['hstrade_combi', 'import/eksport', 'land', 'statistikkvariabel', 'måned',
       'value']]

In [19]:
hstrade_new.tail(10)

,hstrade_combi,import/eksport,land,statistikkvariabel,måned,value
80408,"99603000_2023 VOEC-forsendelser, varer med ver...",Import,USA,Mengde 2 (M2),2025M09,0
80409,"99603000_2023 VOEC-forsendelser, varer med ver...",Eksport,USA,Mengde 1 (M1),2025M09,0
80410,"99603000_2023 VOEC-forsendelser, varer med ver...",Eksport,USA,Verdi (kr),2025M09,0
80411,"99603000_2023 VOEC-forsendelser, varer med ver...",Eksport,USA,Mengde 2 (M2),2025M09,0
80412,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Mengde 1 (M1),2025M09,8592609
80413,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Verdi (kr),2025M09,34056104
80414,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Mengde 2 (M2),2025M09,0
80415,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Mengde 1 (M1),2025M09,68405885
80416,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Verdi (kr),2025M09,482185095
80417,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Mengde 2 (M2),2025M09,0


Endrer kolonne bredde. 

In [21]:
pd.set_option('display.max_colwidth',140)

In [22]:
hstrade_new.tail(10)

,hstrade_combi,import/eksport,land,statistikkvariabel,måned,value
80408,"99603000_2023 VOEC-forsendelser, varer med verdi under 3 000 kr, privatpersoner (2023-) (M1=kg, M2=nei)",Import,USA,Mengde 2 (M2),2025M09,0
80409,"99603000_2023 VOEC-forsendelser, varer med verdi under 3 000 kr, privatpersoner (2023-) (M1=kg, M2=nei)",Eksport,USA,Mengde 1 (M1),2025M09,0
80410,"99603000_2023 VOEC-forsendelser, varer med verdi under 3 000 kr, privatpersoner (2023-) (M1=kg, M2=nei)",Eksport,USA,Verdi (kr),2025M09,0
80411,"99603000_2023 VOEC-forsendelser, varer med verdi under 3 000 kr, privatpersoner (2023-) (M1=kg, M2=nei)",Eksport,USA,Mengde 2 (M2),2025M09,0
80412,"99999999_1988 Sum alle varenumre med restriksjoner ved publisering (1988-) (M1=kg, M2=nei)",Import,USA,Mengde 1 (M1),2025M09,8592609
80413,"99999999_1988 Sum alle varenumre med restriksjoner ved publisering (1988-) (M1=kg, M2=nei)",Import,USA,Verdi (kr),2025M09,34056104
80414,"99999999_1988 Sum alle varenumre med restriksjoner ved publisering (1988-) (M1=kg, M2=nei)",Import,USA,Mengde 2 (M2),2025M09,0
80415,"99999999_1988 Sum alle varenumre med restriksjoner ved publisering (1988-) (M1=kg, M2=nei)",Eksport,USA,Mengde 1 (M1),2025M09,68405885
80416,"99999999_1988 Sum alle varenumre med restriksjoner ved publisering (1988-) (M1=kg, M2=nei)",Eksport,USA,Verdi (kr),2025M09,482185095
80417,"99999999_1988 Sum alle varenumre med restriksjoner ved publisering (1988-) (M1=kg, M2=nei)",Eksport,USA,Mengde 2 (M2),2025M09,0
